### Initiation

In [29]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

In [30]:
# read the words
words = open('names.txt', 'r').read().splitlines()
len(words), words[:5]

(32033, ['emma', 'olivia', 'ava', 'isabella', 'sophia'])

In [31]:
# build the vocabulary
chars = sorted(list(set(''.join(words))))
s_to_i = {'.' : 0} | {s: i + 1 for i, s in enumerate(chars)}
i_to_s = {i: s for s, i in s_to_i.items()}
vocab_size = len(i_to_s)
print(i_to_s)
print(vocab_size)

{0: '.', 1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z'}
27


In [32]:
# build the dataset

block_size = 3 # context length (how many chars to predict next)

def build_dataset(words):
	X, Y = [], []

	for w in words:
		context = [0] * block_size
		for ch in w + '.':
			idx = s_to_i[ch]
			X.append(context)
			Y.append(idx)
			context = context[1:] + [idx]
	
	X = torch.tensor(X)
	Y = torch.tensor(Y)
	print(X.shape, Y.shape)
	return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(len(words) * 0.8)
n2 = int(len(words) * 0.9)

X_train,	Y_train	= build_dataset(words[:n1])		# 80%
X_val,		Y_val	= build_dataset(words[n1:n2])	# 10%
X_test,		Y_test	= build_dataset(words[n2:])		# 10%

torch.Size([182625, 3]) torch.Size([182625])
torch.Size([22655, 3]) torch.Size([22655])
torch.Size([22866, 3]) torch.Size([22866])


In [33]:
# util function used for comparing manual gradients to PyTorch's autograd
def cmp(s, dt, t):
	ex = torch.all(dt == t.grad).item()
	app = torch.allclose(dt, t.grad)
	maxdiff = (dt - t.grad).abs().max().item()
	pad = 1 if app else 10
	print(f'{s:15s} | exact: {str(ex):5s} | approx: {str(app):5s} | max diff: {maxdiff:.{pad}f}')

In [34]:
n_embd = 10 # embedding size
n_hidden = 64 # hidden layer size

g = torch.Generator().manual_seed(2147483647)
C = torch.randn((vocab_size, n_embd),				generator=g)
# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden),	generator=g) * (5/3) / ((n_embd * block_size)**0.5)
b1 = torch.randn((n_hidden,),						generator=g) * 0.1 # for fun (useless)
# Layer 2
W2 = torch.randn((n_hidden, vocab_size),			generator=g) * 0.1
b2 = torch.randn((vocab_size,),						generator=g) * 0.1

# BatchNorm parameters
bngain = torch.randn((1, n_hidden)) * 0.1 + 1.0
bnbias = torch.randn((1, n_hidden)) * 0.1

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
	p.requires_grad = True

4137


In [35]:
batch_size = 32
n = batch_size # a short var for convenience
# construct minibatch
ix = torch.randint(0, X_train.shape[0], (batch_size,), generator=g)
Xb, Yb = X_train[ix], Y_train[ix] # batch X, Y

In [36]:
# forward pass that's "chunkated" into smaller steps to backward one at a time

emb = C[Xb] # embed the chars into vectors
embcat = emb.view(emb.shape[0], -1) # concat the vectors

# Linear layer 1
hprebn = embcat @ W1 + b1 # hidden layer preactivation

# BatchNorm layer
bnmeani = 1 / n * hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff**2
bnvar = 1 / (n - 1) * bndiff2.sum(0, keepdim=True)
bnvar_inv = (bnvar + 1e-5)**-0.5
bnraw = bndiff * bnvar_inv
hpreact = bngain * bnraw + bnbias

# Non-linearity
h = torch.tanh(hpreact) # hidden layer activation

# Linear layer 2
logits = h @ W2 + b2 # output layer

# cross entropy loss
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdim=True)
counts_sum_inv = counts_sum**-1
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(n), Yb].mean()

# PyTorch backward pass
for p in parameters:
	p.grad = None
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv, norm_logits, logit_maxes, logits, h, hpreact, bnraw, bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani, embcat, emb]:
	t.retain_grad()
loss.backward()
loss

tensor(3.3273, grad_fn=<NegBackward0>)

In [37]:
logprobs.shape, logprobs[range(n), Yb], Yb.shape

(torch.Size([32, 27]),
 tensor([-4.0869, -3.0877, -3.6153, -3.2970, -4.1304, -3.3986, -3.1446, -3.9603,
         -3.0718, -4.2444, -3.2318, -1.6564, -2.9673, -2.9929, -2.9785, -3.1066,
         -3.9200, -2.8471, -3.6825, -3.3317, -2.8584, -2.9320, -4.3010, -4.0060,
         -3.5375, -2.8370, -2.9669, -3.9421, -2.6241, -3.3688, -3.2364, -3.1123],
        grad_fn=<IndexBackward0>),
 torch.Size([32]))

In [47]:
dlogprobs = torch.zeros_like(logprobs)
dlogprobs[range(n), Yb] = -1.0 / n

dprobs = (1.0 / probs) * dlogprobs

dcounts_sum_inv = (counts * dprobs).sum(1, keepdim=True)
dcounts = counts_sum_inv * dprobs
dcounts_sum = (-counts_sum**-2) * dcounts_sum_inv
dcounts += torch.ones_like(counts) * dcounts_sum
dnorm_logits = counts * dcounts
dlogits = dnorm_logits.clone()
dlogit_maxes = (-dnorm_logits).sum(1, keepdim=True)
dlogits += F.one_hot(logits.max(1).indices, num_classes=logits.shape[1]) * dlogit_maxes
dh = dlogits @ W2.T
dW2 = h.T @ dlogits
db2 = dlogits.sum(0)
dhpreact = (1.0 - h**2) * dh
dbngain = (bnraw * dhpreact).sum(0, keepdim=True)
dbnraw = bngain * dhpreact
dbnbias = dhpreact.sum(0, keepdim=True)
dbndiff = bnvar_inv * dbnraw
dbnvar_inv = (bndiff * dbnraw).sum(0, keepdim=True)
dbnvar = (-0.5 * (bnvar + 1e-5)**-1.5) * dbnvar_inv
dbndiff2 = (1.0 / (n - 1)) * torch.ones_like(bndiff2) * dbnvar
dbndiff += (2 * bndiff) * dbndiff2 
dhprebn = dbndiff.clone()
dbnmeani = (-dbndiff).sum(0)
dhprebn += 1.0/n * (torch.ones_like(hprebn) * dbnmeani)
dembcat = dhprebn @ W1.T
dW1 = embcat.T @ dhprebn
db1 = dhprebn.sum(0)
demb = dembcat.view(emb.shape)
dC = torch.zeros_like(C)
for k in range(Xb.shape[0]):
	for j in range(Xb.shape[1]):
		dC[Xb[k, j]] += demb[k, j]

cmp('logprobs', dlogprobs, logprobs)
cmp('probs', dprobs, probs)
cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)
cmp('counts_sum', dcounts_sum, counts_sum)
cmp('counts', dcounts, counts)
cmp('norm_logits', dnorm_logits, norm_logits)
cmp('logit_maxes', dlogit_maxes, logit_maxes)
cmp('logits', dlogits, logits)
cmp('h', dh, h)
cmp('W2', dW2, W2)
cmp('b2', db2, b2)
cmp('hpreact', dhpreact, hpreact)
cmp('bngain', dbngain, bngain)
cmp('bnbias', dbnbias, bnbias)
cmp('bnraw', dbnraw, bnraw)
cmp('bnvar_inv', dbnvar_inv, bnvar_inv)
cmp('bnvar', dbnvar, bnvar)
cmp('bndiff2', dbndiff2, bndiff2)
cmp('bndiff', dbndiff, bndiff)
cmp('bnmeani', dbnmeani, bnmeani)
cmp('hprebn', dhprebn, hprebn)
cmp('embcat', dembcat, embcat)
cmp('W1', dW1, W1)
cmp('b1', db1, b1)
cmp('emb', demb, emb)
cmp('C', dC, C)

logprobs        | exact: True  | approx: True  | max diff: 0.0
probs           | exact: True  | approx: True  | max diff: 0.0
counts_sum_inv  | exact: True  | approx: True  | max diff: 0.0
counts_sum      | exact: True  | approx: True  | max diff: 0.0
counts          | exact: True  | approx: True  | max diff: 0.0
norm_logits     | exact: True  | approx: True  | max diff: 0.0
logit_maxes     | exact: True  | approx: True  | max diff: 0.0
logits          | exact: True  | approx: True  | max diff: 0.0
h               | exact: True  | approx: True  | max diff: 0.0
W2              | exact: True  | approx: True  | max diff: 0.0
b2              | exact: True  | approx: True  | max diff: 0.0
hpreact         | exact: False | approx: True  | max diff: 0.0
bngain          | exact: False | approx: True  | max diff: 0.0
bnbias          | exact: False | approx: True  | max diff: 0.0
bnraw           | exact: False | approx: True  | max diff: 0.0
bnvar_inv       | exact: False | approx: True  | max di

In [48]:
# easier backward
loss_fast = F.cross_entropy(logits, Yb)
print(loss_fast.item(), 'diff:', (loss_fast - loss).item())

3.32732892036438 diff: 4.76837158203125e-07


In [51]:
# backpass

dlogits = F.softmax(logits, 1)
dlogits[range(n), Yb] -= 1.0
dlogits /= n

cmp('logits', dlogits, logits)

logits          | exact: False | approx: True  | max diff: 0.0


In [53]:
# easier forward
# hpreact_fast = bngain * (hprebn - hprebn.mean(0, keepdim=True)) / (hprebn.var(0, unbiased=False, keepdim=True) + 1e-5).sqrt() + bnbias
# print('max diff:', (hpreact_fast - hpreact).abs().max().item())

In [54]:
# backpass

dhprebn = bngain * bnvar_inv / n * (n * dhpreact - dhpreact.sum(0) - n/(n-1) * bnraw * (dhpreact * bnraw).sum(0))

cmp('hprebn', dhprebn, hprebn)

hprebn          | exact: False | approx: True  | max diff: 0.0


In [62]:
# all together!

n_embd = 10 # embedding size
n_hidden = 200 # hidden layer size

g = torch.Generator().manual_seed(2147483647)
C = torch.randn((vocab_size, n_embd),				generator=g)
# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden),	generator=g) * (5/3) / ((n_embd * block_size)**0.5)
b1 = torch.randn((n_hidden,),						generator=g) * 0.1 # for fun (useless)
# Layer 2
W2 = torch.randn((n_hidden, vocab_size),			generator=g) * 0.1
b2 = torch.randn((vocab_size,),						generator=g) * 0.1

# BatchNorm parameters
bngain = torch.randn((1, n_hidden)) * 0.1 + 1.0
bnbias = torch.randn((1, n_hidden)) * 0.1

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
	p.requires_grad = True

max_steps = 200_000
batch_size = 32
n = batch_size # a short var for convenience
lossi = []


with torch.no_grad():
	# optimization
	for i in range(max_steps):
		# minibatch
		ix = torch.randint(0, X_train.shape[0], (batch_size,), generator=g)
		Xb, Yb = X_train[ix], Y_train[ix] # batch X, Y

		# forward pass
		emb = C[Xb] # embed the chars into vectors
		embcat = emb.view(emb.shape[0], -1) # concat the vectors

		# Linear layer 1
		hprebn = embcat @ W1 + b1 # hidden layer preactivation

		# BatchNorm layer
		# --------------------------------
		bnmean = hprebn.mean(0, keepdim=True)
		bnvar = hprebn.var(0, unbiased=True, keepdim=True)
		bnvar_inv = (bnvar + 1e-5)**-0.5
		bnraw = (hprebn - bnmean) * bnvar_inv
		hpreact = bngain * bnraw + bnbias
		# --------------------------------

		# Non-linearity
		h = torch.tanh(hpreact) # hidden layer activation
		logits = h @ W2 + b2 # output layer
		loss = F.cross_entropy(logits, Yb) # loss function

		# backward pass
		for p in parameters:
			p.grad = None
		# loss.backward()

		# manual backprop!
		# --------------------------------
		dlogits = F.softmax(logits, 1)
		dlogits[range(n), Yb] -= 1.0
		dlogits /= n
		# 2nd layer backprop
		dh = dlogits @ W2.T
		dW2 = h.T @ dlogits
		db2 = dlogits.sum(0)
		# tanh
		dhpreact = (1.0 - h**2) * dh
		# batchnorm backprop
		dbngain = (bnraw * dhpreact).sum(0, keepdim=True)
		dbnbias = dhpreact.sum(0, keepdim=True)
		dhprebn = bngain * bnvar_inv / n * (n * dhpreact - dhpreact.sum(0) - n/(n-1) * bnraw * (dhpreact * bnraw).sum(0))
		# 1st layer backprop
		dembcat = dhprebn @ W1.T
		dW1 = embcat.T @ dhprebn
		db1 = dhprebn.sum(0)
		# embedding backprop
		demb = dembcat.view(emb.shape)
		dC = torch.zeros_like(C)
		for k in range(Xb.shape[0]):
			for j in range(Xb.shape[1]):
				dC[Xb[k, j]] += demb[k, j]
		grads = [dC, dW1, db1, dW2, db2, dbngain, dbnbias]
		# --------------------------------

		# update
		lr = 0.1 if i < 100_000 else 0.01
		for p, grad in zip(parameters, grads):
			# p.data += -lr * p.grad
			p.data += -lr * grad # NEW WAY LET'S GO
		
		if i % 10000 == 0:
			print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')
		lossi.append(loss.log10().item())

		# if i >= 100:
		# 	break

12297
      0/ 200000: 3.7786
  10000/ 200000: 2.2112
  20000/ 200000: 2.3715
  30000/ 200000: 2.4359
  40000/ 200000: 2.0316
  50000/ 200000: 2.3260
  60000/ 200000: 2.4419
  70000/ 200000: 2.0399
  80000/ 200000: 2.4143
  90000/ 200000: 2.0988
 100000/ 200000: 2.0461
 110000/ 200000: 2.2988
 120000/ 200000: 2.0005
 130000/ 200000: 2.3717
 140000/ 200000: 2.2165
 150000/ 200000: 2.1558
 160000/ 200000: 2.0320
 170000/ 200000: 1.8355
 180000/ 200000: 2.0073
 190000/ 200000: 1.9156


In [60]:
# check gradients
# for p, g in zip(parameters, grads):
# 	cmp(str(tuple(p.shape)), g, p)

(27, 10)        | exact: False | approx: True  | max diff: 0.0
(30, 200)       | exact: False | approx: True  | max diff: 0.0
(200,)          | exact: False | approx: True  | max diff: 0.0
(200, 27)       | exact: False | approx: True  | max diff: 0.0
(27,)           | exact: False | approx: True  | max diff: 0.0
(1, 200)        | exact: False | approx: True  | max diff: 0.0
(1, 200)        | exact: False | approx: True  | max diff: 0.0


In [63]:
with torch.no_grad():
	emb = C[X_train]
	embcat = emb.view(emb.shape[0], -1)
	hprebn = embcat @ W1 + b1
	bnmean = hprebn.mean(0, keepdim=True)
	bnvar = hprebn.var(0, unbiased=True, keepdim=True)

In [64]:
@torch.no_grad()
def split_loss(split):
	x, y = {
		'train': (X_train, Y_train),
		'val': (X_val, Y_val),
		'test': (X_test, Y_test)
	}[split]
	emb = C[x]
	embcat = emb.view(emb.shape[0], -1)
	hpreact = embcat @ W1 + b1
	hpreact = bngain * (hpreact - bnmean) * (bnvar + 1e-5)**-0.5 + bnbias
	h = torch.tanh(hpreact)
	logits = h @ W2 + b2
	loss = F.cross_entropy(logits, y)
	print(split, loss.item())

split_loss('train')
split_loss('val')

train 2.0718963146209717
val 2.111926794052124


In [66]:
g = torch.Generator().manual_seed(2147483647 + 10)

for _ in range(20):
	out = []
	context = [0] * block_size
	while True:
		emb = C[torch.tensor([context])]
		embcat = emb.view(emb.shape[0], -1)
		hpreact = embcat @ W1 + b1
		hpreact = bngain * (hpreact - bnmean) * (bnvar + 1e-5)**-0.5 + bnbias
		h = torch.tanh(hpreact)
		logits = h @ W2 + b2
		# Sample
		probs = F.softmax(logits, dim=1)
		ix = torch.multinomial(probs, num_samples=1, generator=g).item()
		context = context[1:] + [ix]
		out.append(ix)
		if ix == 0:
			break
	
	print(''.join(i_to_s[i] for i in out[:-1]))

mora
mayah
see
mad
ryla
rethruthadraeg
adelynnelin
shi
jen
eden
sana
arleigh
malaia
nosalbergihimie
tricke
jennex
teron
ubrence
ryyah
faeh
